In [75]:
import os, json
import pandas as pd

# this finds our json files
path_to_json = 'F:/kalu/BITS/DeepLearning/assignment1/CORD/train/json/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# here I define my pandas Dataframe with the columns I want to get from the json
jsons_data = pd.DataFrame(columns=['validline'])

f = open("F:/kalu/BITS/DeepLearning/assignment1/receipt_words.csv", "a")
print("filename",",","index",",","x1",",",'x2',",",'x3',",",'x4',",",'y1',",",'y2',",",'y3',",",'y4',",",'is_key',",",'row_id',",",'text',",",'category',",","group_id",file=f)


# we need both the json and an index number so use enumerate()
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)
        
       # print(json_text)

        # here you need to know the layout of your json and each json has to have
        # the same structure (obviously not the structure I have here)
        lines = json_text['valid_line']
        
        for x in lines:
            words = x['words']
            category = x['category']
            group_id = x['group_id']
            for y in words:
                print(js,index,y['quad']['x1'],y['quad']['x2'],y['quad']['x3'],y['quad']['x4'],y['quad']['y1'],y['quad']['y2'],y['quad']['y3'],y['quad']['y4'],y['is_key'],y['row_id'],"\""+y['text']+"\"",category,group_id,file=f,sep=",")

       # city = json_text['features'][0]['properties']['name']
        #lonlat = json_text['features'][0]['geometry']['coordinates']
        # here I push a list of data into a pandas DataFrame at row given by 'index'
        #f.close()
        #jsons_data.loc[index] = [words, city, lonlat]

# now that we have the pertinent json data in our DataFrame let's look at it
f.close()

In [1]:
import pandas as pd
receipts_df = pd.read_csv("F:/kalu/BITS/DeepLearning/assignment1/receipt_words.csv")
receipts_df.columns = receipts_df.columns.str.strip()
receipts_df['text'] = receipts_df['text'].str.strip()


In [2]:
receipts_df.dtypes

filename    object
index        int64
x1           int64
x2           int64
x3           int64
x4           int64
y1           int64
y2           int64
y3           int64
y4           int64
is_key       int64
row_id       int64
text        object
category    object
group_id     int64
dtype: object

In [3]:
char2idx = {"START":0,"END":1,"a":2,"b":3,"c":4,"d":5,"e":6,"f":7,"g":8,"h":9,"i":10,"j":11,"k":12,"l":13,"m":14,"n":15,"o":16,"p":17,"q":18,"r":19,"s":20,"t":21,"u":22,"v":23,"w":24,"x":25,"y":26,"z":27,"0":28,"1":29,"2":30,"3":31,"4":32,"5":33,"6":34,"7":35,"8":36,"9":37,".":38,"-":39,",":40,":":41," ":42}

In [4]:
#p_test.SentimentText=p_test.SentimentText.astype(str)
receipts_df.text = receipts_df.text.astype(str)
receipts_df.dtypes

filename    object
index        int64
x1           int64
x2           int64
x3           int64
x4           int64
y1           int64
y2           int64
y3           int64
y4           int64
is_key       int64
row_id       int64
text        object
category    object
group_id     int64
dtype: object

In [5]:
words = receipts_df['text']

In [6]:
indexed_words = []
j=43
for word in words:
    indexed_word = []
    #print(word)
    word = word.lower();
    for i in range(len(word)):
        token = word[i]
        if token not in char2idx:
            char2idx[token] = j
            j += 1
        indexed_word.append(char2idx[token])
    indexed_words.append(indexed_word)

receipts_df["indexed_words"] = indexed_words;

In [7]:
import numpy as np
def get_bigram_probs(words, V, start_idx, end_idx, smoothing=1):
  # structure of bigram probability matrix will be:
  # (last word, current word) --> probability
  # we will use add-1 smoothing
  # note: we'll always ignore this from the END token
  bigram_probs = np.ones((V, V)) * smoothing
  for word in words:
    for i in range(len(word)):
      
      if i == 0:
        # beginning word
        word[i]
        bigram_probs[start_idx, word[i]] += 1
      else:
        # middle word
        bigram_probs[word[i-1], word[i]] += 1

      # if we're at the final word
      # we update the bigram for last -> current
      # AND current -> END token
      if i == len(word) - 1:
        # final word
        bigram_probs[word[i], end_idx] += 1

  # normalize the counts along the rows to get probabilities
  bigram_probs /= bigram_probs.sum(axis=1, keepdims=True)
  return bigram_probs

In [8]:
start_idx = char2idx['START']
end_idx = char2idx['END']
V = len(char2idx)



In [9]:
bigram_probs = get_bigram_probs(indexed_words, V, start_idx, end_idx, smoothing=0.1)

In [10]:
bigram_probs

array([[4.22357940e-06, 4.22357940e-06, 1.55047600e-02, ...,
        8.86951674e-05, 8.86951674e-05, 8.86951674e-05],
       [1.51515152e-02, 1.51515152e-02, 1.51515152e-02, ...,
        1.51515152e-02, 1.51515152e-02, 1.51515152e-02],
       [1.26412661e-05, 7.75036028e-02, 6.44704574e-04, ...,
        1.26412661e-05, 1.26412661e-05, 1.26412661e-05],
       ...,
       [1.16279070e-02, 2.44186047e-01, 1.16279070e-02, ...,
        1.16279070e-02, 1.16279070e-02, 1.16279070e-02],
       [1.16279070e-02, 1.16279070e-02, 1.27906977e-01, ...,
        1.16279070e-02, 1.16279070e-02, 1.16279070e-02],
       [1.16279070e-02, 2.44186047e-01, 1.16279070e-02, ...,
        1.16279070e-02, 1.16279070e-02, 1.16279070e-02]])

In [11]:
 # a function to calculate normalized log prob score
  # for a sentence
def get_score(word):
    #print("word")
    #print(word)
    score = 0
    if len(word) > 0 :
        for i in range(len(word)):
          if i == 0:
            # beginning word
            score += np.log(bigram_probs[start_idx, word[i]])
          else:
            # middle word
            score += np.log(bigram_probs[word[i-1], word[i]])
        # final word
        score += np.log(bigram_probs[word[i-1], end_idx])

    # normalize the score
    # return score / (len(word) + 1)
    if score == 0:
        return score;
    else :
        return  ((-1)*(len(word) + 1)/score)*100

In [12]:
from future.utils import iteritems

idx2char = dict((v, k) for k, v in iteritems(char2idx))
def get_word(word):
    return ''.join(idx2char[i] for i in word)

In [13]:
receipts_df['text_score'] = receipts_df.apply(lambda row : get_score(row['indexed_words']), axis = 1)

In [14]:
receipts_df

,filename,index,x1,x2,x3,x4,y1,y2,y3,y4,is_key,row_id,text,category,group_id,indexed_words,text_score
0,receipt_00000.json,0,256,270,270,256,374,374,390,390,0,2179893,x,menu.cnt,3,[25],39.130296
1,receipt_00000.json,0,258,270,270,258,402,402,418,418,0,2179894,x,menu.cnt,4,[25],39.130296
2,receipt_00000.json,0,258,272,272,258,428,428,444,444,0,2179895,x,menu.cnt,5,[25],39.130296
3,receipt_00000.json,0,260,274,274,260,456,456,470,470,0,2179896,x,menu.cnt,6,[25],39.130296
4,receipt_00000.json,0,258,274,274,258,480,480,496,496,0,2179897,x,menu.cnt,7,[25],39.130296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23665,receipt_00099.json,99,224,275,277,226,1064,1060,1086,1089,1,2242955,NON,total.creditcardprice,5,"[15, 16, 15]",29.891933
23666,receipt_00099.json,99,276,338,338,276,1060,1060,1086,1086,1,2242955,TUNAI,total.creditcardprice,5,"[21, 22, 15, 2, 10]",36.201937
23667,receipt_00099.json,99,450,515,513,449,1053,1056,1083,1080,0,2242955,"75,000",total.creditcardprice,5,"[35, 33, 40, 28, 28, 28]",60.294739
23668,receipt_00099.json,99,224,313,315,226,1090,1086,1112,1117,1,2242956,KEMBALI,total.changeprice,5,"[12, 6, 14, 3, 2, 13, 10]",49.614100


In [15]:
categories = receipts_df['category'] 
category2idx = {}
idx2category = {}
j=0;
for category in categories:
    if category not in category2idx:
        category2idx[category] = j
        idx2category[j] = category
        j+=1

category2idx

def get_index_for_category(category):
    return category2idx[category]


def get_category_for_index(index):
    return idx2category[index]
        

In [16]:
receipts_df['category_idx'] = receipts_df.apply(lambda row : get_index_for_category(row['category']), axis = 1)
category2idx

{'menu.cnt': 0,
 'menu.nm': 1,
 'menu.price': 2,
 'sub_total.subtotal_price': 3,
 'sub_total.service_price': 4,
 'sub_total.tax_price': 5,
 'sub_total.etc': 6,
 'total.total_price': 7,
 'menu.sub_nm': 8,
 'total.cashprice': 9,
 'total.changeprice': 10,
 'total.menutype_cnt': 11,
 'total.menuqty_cnt': 12,
 'sub_total.discount_price': 13,
 'menu.unitprice': 14,
 'total.creditcardprice': 15,
 'menu.num': 16,
 'menu.sub_cnt': 17,
 'menu.discountprice': 18,
 'total.total_etc': 19,
 'menu.sub_price': 20,
 'total.emoneyprice': 21,
 'menu.sub_unitprice': 22,
 'menu.sub_etc': 23,
 'void_menu.nm': 24,
 'void_menu.price': 25,
 'sub_total.othersvc_price': 26,
 'menu.vatyn': 27,
 'menu.itemsubtotal': 28,
 'menu.etc': 29}

In [17]:
data = receipts_df[['x1','x2','x3','x4','y1','y2','y3','y4','is_key','text_score','category_idx','index']]

import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
x = data.values #returns a numpy array
#min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler = preprocessing.StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
print(x_scaled.shape)
X = pd.DataFrame(x_scaled,columns=['x1','x2','x3','x4','y1','y2','y3','y4','is_key','text_score','category_idx','index'])
X

(23670, 12)


,x1,x2,x3,x4,y1,y2,y3,y4,is_key,text_score,category_idx,index
0,-0.448185,-0.618908,-0.619223,-0.448084,-0.914594,-0.914259,-0.925631,-0.925978,-0.572895,-0.598368,-0.984142,-1.326901
1,-0.442199,-0.618908,-0.619223,-0.442097,-0.866315,-0.865981,-0.878641,-0.878992,-0.572895,-0.598368,-0.984142,-1.326901
2,-0.442199,-0.613557,-0.613872,-0.442097,-0.821485,-0.821152,-0.835008,-0.835362,-0.572895,-0.598368,-0.984142,-1.326901
3,-0.436213,-0.608205,-0.608522,-0.436110,-0.773206,-0.772874,-0.791374,-0.791732,-0.572895,-0.598368,-0.984142,-1.326901
4,-0.442199,-0.608205,-0.608522,-0.442097,-0.731825,-0.731493,-0.747741,-0.748102,-0.572895,-0.598368,-0.984142,-1.326901
...,...,...,...,...,...,...,...,...,...,...,...,...
23665,-0.543962,-0.605529,-0.600495,-0.537887,0.275128,0.268544,0.242399,0.246993,1.745522,-1.196121,1.900263,-0.926002
23666,-0.388324,-0.436956,-0.437296,-0.388215,0.268231,0.268544,0.242399,0.241958,1.745522,-0.787842,1.900263,-0.926002
23667,0.132466,0.036657,0.030900,0.129650,0.256161,0.261647,0.237365,0.231890,-0.572895,0.771043,1.900263,-0.926002
23668,-0.543962,-0.503850,-0.498830,-0.537887,0.319958,0.313373,0.286033,0.293979,1.745522,0.079970,0.938795,-0.926002


In [18]:


from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)

from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance

def getcosine(X,Y):
    return cosine_similarity(X,Y)[0,0];

def getneighourscore(index,X):
    return X['text_score'].tolist()[0];

X["n1_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,1]]]), axis = 1)
print("n1_cd done")
X["n2_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,2]]]), axis = 1)
print("n2_cd done")
X["n3_cd"] = X.apply(lambda row : getcosine(X.loc[[row.name]],X.loc[[indices[row.name,3]]]), axis = 1)
print("n3_cd done")

X["n1_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,1]]]), axis = 1)
print("n1_ed done")
X["n2_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,2]]]), axis = 1)
print("n2_ed done")
X["n3_ed"] = X.apply(lambda row : distance.euclidean(X.loc[[row.name]],X.loc[[indices[row.name,3]]]), axis = 1)
print("n3_ed done")

X["n1_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,1]]]), axis = 1)
print("n1_score done")
X["n2_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,2]]]), axis = 1)
print("n2_score done")
X["n3_score"] = X.apply(lambda row : getneighourscore(row.name,X.loc[[indices[row.name,3]]]), axis = 1)
print("n3_score done")

X

n1_cd done
n2_cd done
n3_cd done
n1_ed done
n2_ed done
n3_ed done
n1_score done
n2_score done
n3_score done


,x1,x2,x3,x4,y1,y2,y3,y4,is_key,text_score,...,index,n1_cd,n2_cd,n3_cd,n1_ed,n2_ed,n3_ed,n1_score,n2_score,n3_score
0,-0.448185,-0.618908,-0.619223,-0.448084,-0.914594,-0.914259,-0.925631,-0.925978,-0.572895,-0.598368,...,-1.326901,0.999668,0.998781,0.997783,0.095660,0.184227,0.251241,-0.598368,-0.598368,-0.638298
1,-0.442199,-0.618908,-0.619223,-0.442097,-0.866315,-0.865981,-0.878641,-0.878992,-0.572895,-0.598368,...,-1.326901,0.999693,0.999680,0.998813,0.088792,0.095906,0.181354,-0.598368,-0.598368,-0.598368
2,-0.442199,-0.613557,-0.613872,-0.442097,-0.821485,-0.821152,-0.835008,-0.835362,-0.572895,-0.598368,...,-1.326901,0.999693,0.999665,0.998756,0.088792,0.092785,0.177167,-0.598368,-0.598368,-0.598368
3,-0.436213,-0.608205,-0.608522,-0.436110,-0.773206,-0.772874,-0.791374,-0.791732,-0.572895,-0.598368,...,-1.326901,0.999631,0.999665,0.998725,0.085463,0.092785,0.173719,-0.598368,-0.598368,-0.598368
4,-0.442199,-0.608205,-0.608522,-0.442097,-0.731825,-0.731493,-0.747741,-0.748102,-0.572895,-0.598368,...,-1.326901,0.999631,0.999643,0.998446,0.085463,0.089274,0.200649,-0.598368,-0.598368,-0.686272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23665,-0.543962,-0.605529,-0.600495,-0.537887,0.275128,0.268544,0.242399,0.246993,1.745522,-1.196121,...,-0.926002,1.000000,0.996847,0.993807,0.000000,0.378269,0.487362,-1.196121,-1.300113,-1.291323
23666,-0.388324,-0.436956,-0.437296,-0.388215,0.268231,0.268544,0.242399,0.241958,1.745522,-0.787842,...,-0.926002,1.000000,0.996294,0.994255,0.000000,0.390122,0.521701,-0.787842,-0.739529,-0.822781
23667,0.132466,0.036657,0.030900,0.129650,0.256161,0.261647,0.237365,0.231890,-0.572895,0.771043,...,-0.926002,1.000000,0.983385,0.985471,0.000000,0.473091,0.473091,0.771043,0.495102,0.495102
23668,-0.543962,-0.503850,-0.498830,-0.537887,0.319958,0.313373,0.286033,0.293979,1.745522,0.079970,...,-0.926002,1.000000,0.993901,0.993777,0.000000,0.425225,0.455084,0.079970,0.173901,0.173901


In [19]:
receipts_df['category_idx'] 

0         0
1         0
2         0
3         0
4         0
         ..
23665    15
23666    15
23667    15
23668    10
23669    10
Name: category_idx, Length: 23670, dtype: int64

In [20]:
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras import layers

In [21]:
X

,x1,x2,x3,x4,y1,y2,y3,y4,is_key,text_score,...,index,n1_cd,n2_cd,n3_cd,n1_ed,n2_ed,n3_ed,n1_score,n2_score,n3_score
0,-0.448185,-0.618908,-0.619223,-0.448084,-0.914594,-0.914259,-0.925631,-0.925978,-0.572895,-0.598368,...,-1.326901,0.999668,0.998781,0.997783,0.095660,0.184227,0.251241,-0.598368,-0.598368,-0.638298
1,-0.442199,-0.618908,-0.619223,-0.442097,-0.866315,-0.865981,-0.878641,-0.878992,-0.572895,-0.598368,...,-1.326901,0.999693,0.999680,0.998813,0.088792,0.095906,0.181354,-0.598368,-0.598368,-0.598368
2,-0.442199,-0.613557,-0.613872,-0.442097,-0.821485,-0.821152,-0.835008,-0.835362,-0.572895,-0.598368,...,-1.326901,0.999693,0.999665,0.998756,0.088792,0.092785,0.177167,-0.598368,-0.598368,-0.598368
3,-0.436213,-0.608205,-0.608522,-0.436110,-0.773206,-0.772874,-0.791374,-0.791732,-0.572895,-0.598368,...,-1.326901,0.999631,0.999665,0.998725,0.085463,0.092785,0.173719,-0.598368,-0.598368,-0.598368
4,-0.442199,-0.608205,-0.608522,-0.442097,-0.731825,-0.731493,-0.747741,-0.748102,-0.572895,-0.598368,...,-1.326901,0.999631,0.999643,0.998446,0.085463,0.089274,0.200649,-0.598368,-0.598368,-0.686272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23665,-0.543962,-0.605529,-0.600495,-0.537887,0.275128,0.268544,0.242399,0.246993,1.745522,-1.196121,...,-0.926002,1.000000,0.996847,0.993807,0.000000,0.378269,0.487362,-1.196121,-1.300113,-1.291323
23666,-0.388324,-0.436956,-0.437296,-0.388215,0.268231,0.268544,0.242399,0.241958,1.745522,-0.787842,...,-0.926002,1.000000,0.996294,0.994255,0.000000,0.390122,0.521701,-0.787842,-0.739529,-0.822781
23667,0.132466,0.036657,0.030900,0.129650,0.256161,0.261647,0.237365,0.231890,-0.572895,0.771043,...,-0.926002,1.000000,0.983385,0.985471,0.000000,0.473091,0.473091,0.771043,0.495102,0.495102
23668,-0.543962,-0.503850,-0.498830,-0.537887,0.319958,0.313373,0.286033,0.293979,1.745522,0.079970,...,-0.926002,1.000000,0.993901,0.993777,0.000000,0.425225,0.455084,0.079970,0.173901,0.173901


In [22]:
X.shape

(23670, 21)

In [24]:
X.columns.values

array(['x1', 'x2', 'x3', 'x4', 'y1', 'y2', 'y3', 'y4', 'is_key',
       'text_score', 'category_idx', 'index', 'n1_cd', 'n2_cd', 'n3_cd',
       'n1_ed', 'n2_ed', 'n3_ed', 'n1_score', 'n2_score', 'n3_score'],
      dtype=object)

In [102]:
XX=X
XX['New_ID'] = range(0, 0+len(XX))
type(XX.columns.values)
cols = np.delete(XX.columns.values, np.where(XX.columns.values == 'New_ID'))
cols
Y=XX.groupby(['New_ID']).apply(lambda subdf : subdf[cols].values).reset_index()
Y[0]
Y[0].values
Z=Y[0].to_numpy()
Z.shape
Z[0]
A=np.stack(Z)
B=A.transpose(0,2,1)
B.shape

(23670, 21, 1)

array(['x1', 'x2', 'x3', 'x4', 'y1', 'y2', 'y3', 'y4', 'is_key',
       'text_score', 'category_idx', 'index', 'n1_cd', 'n2_cd', 'n3_cd',
       'n1_ed', 'n2_ed', 'n3_ed', 'n1_score', 'n2_score', 'n3_score'],
      dtype=object)

0        [[-0.4481849528394153, -0.6189083480312535, -0...
1        [[-0.4421988655920413, -0.6189083480312535, -0...
2        [[-0.4421988655920413, -0.6135567979629325, -0...
3        [[-0.43621277834466726, -0.6082052478946115, -...
4        [[-0.4421988655920413, -0.6082052478946115, -0...
                               ...                        
23665    [[-0.5439623487973994, -0.6055294728604511, -0...
23666    [[-0.3883240803656753, -0.43695564570834, -0.4...
23667    [[0.13246551015586303, 0.03665653533806731, 0....
23668    [[-0.5439623487973994, -0.5038500215623524, -0...
23669    [[0.2761316040928391, 0.033980760303906815, 0....
Name: 0, Length: 23670, dtype: object

array([array([[-0.44818495, -0.61890835, -0.61922322, -0.4480836 , -0.91459367,
        -0.91425893, -0.92563073, -0.92597761, -0.57289466, -0.59836756,
        -0.98414245, -1.32690114,  0.99966824,  0.99878104,  0.99778304,
         0.09565985,  0.18422729,  0.25124068, -0.59836756, -0.59836756,
        -0.63829773]]),
       array([[-0.44219887, -0.61890835, -0.61922322, -0.44209672, -0.86631512,
        -0.86598127, -0.87864102, -0.87899168, -0.57289466, -0.59836756,
        -0.98414245, -1.32690114,  0.99969309,  0.99968018,  0.99881274,
         0.08879244,  0.09590604,  0.18135437, -0.59836756, -0.59836756,
        -0.59836756]]),
       array([[-0.44219887, -0.6135568 , -0.61387241, -0.44209672, -0.82148503,
        -0.82115201, -0.83500772, -0.83536188, -0.57289466, -0.59836756,
        -0.98414245, -1.32690114,  0.99969309,  0.99966461,  0.99875648,
         0.08879244,  0.09278475,  0.17716678, -0.59836756, -0.59836756,
        -0.59836756]]),
       ...,
       array([[ 0.1

array([[-0.44818495, -0.61890835, -0.61922322, -0.4480836 , -0.91459367,
        -0.91425893, -0.92563073, -0.92597761, -0.57289466, -0.59836756,
        -0.98414245, -1.32690114,  0.99966824,  0.99878104,  0.99778304,
         0.09565985,  0.18422729,  0.25124068, -0.59836756, -0.59836756,
        -0.63829773]])

In [93]:
A.shape

(23670, 1, 21)

In [94]:
B=A.transpose(0,2,1)

In [95]:
B.shape

(23670, 21, 1)

In [96]:
B

array([[[-0.44818495],
        [-0.61890835],
        [-0.61922322],
        ...,
        [-0.59836756],
        [-0.59836756],
        [-0.63829773]],

       [[-0.44219887],
        [-0.61890835],
        [-0.61922322],
        ...,
        [-0.59836756],
        [-0.59836756],
        [-0.59836756]],

       [[-0.44219887],
        [-0.6135568 ],
        [-0.61387241],
        ...,
        [-0.59836756],
        [-0.59836756],
        [-0.59836756]],

       ...,

       [[ 0.13246551],
        [ 0.03665654],
        [ 0.03089996],
        ...,
        [ 0.77104269],
        [ 0.4951015 ],
        [ 0.4951015 ]],

       [[-0.54396235],
        [-0.50385002],
        [-0.49883004],
        ...,
        [ 0.07996963],
        [ 0.17390079],
        [ 0.17390079]],

       [[ 0.2761316 ],
        [ 0.03398076],
        [ 0.03357536],
        ...,
        [-0.48194179],
        [-0.48194179],
        [-0.48194179]]])

In [125]:

#data = receipts_df[['x1','x2','x3','x4','y1','y2','y3','y4','is_key','text_score','category_idx','index']]
data = X

data['category_idx']=receipts_df['category_idx'];

#from sklearn.neighbors import NearestNeighbors
#nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)
#distances, indices = nbrs.kneighbors(data[0])


from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)
y = data.pop('category_idx')
X = data

import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
x = X.values #returns a numpy array
#min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler = preprocessing.StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
X = pd.DataFrame(x_scaled)

X_train,X_test,y_train,y_test = train_test_split(X.index,y.index,test_size=0.2)
xtrain=X.iloc[X_train]
xtest=X.iloc[X_test]
y_train=y.iloc[y_train]
y_test=y.iloc[y_test]

from imblearn.over_sampling import RandomOverSampler
from collections import Counter
print('Before Resampled dataset shape' )
print( y_train.shape)
sm = RandomOverSampler('all',random_state=42)
xtrain, y_train = sm.fit_resample(xtrain, y_train)
print(' Resampled dataset shape' )
print( y_train.shape)

ytrain = np.zeros((len(y_train), len(idx2category.values())))
ytest = np.zeros((len(y_test), len(idx2category.values())))
for i in range(len(y_train)):
    #print(y_train.values[i])
    ytrain[i][y_train.values[i]]=1
    
for i in range(len(y_test)):
    #print(y_train.values[i])
    ytrain[i][y_test.values[i]]=1



Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_19 (Conv1D)           (None, 20, 32)            128       
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 10, 32)            0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 8, 64)             6208      
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 6, 64)             12352     
_________________________________________________________________
flatten_5 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 20)                7700      
_________________________________________________________________
dense_11 (Dense)             (None, 30)               

C:\Users\kalus\anaconda3\lib\site-packages\imblearn\utils\_validation.py:638: FutureWarning: Pass sampling_strategy=all as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning)


In [126]:
type(xtrain)

pandas.core.frame.DataFrame

In [127]:
xtrain

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,-1.196446,-0.889162,-0.894790,-1.199437,-0.811140,-0.798737,-0.791374,-0.805157,-0.572895,-0.673646,...,0.480123,0.681038,0.068073,-0.445561,-0.598267,-0.739085,-0.538142,-0.752523,-0.514054,-0.514054
1,0.132466,0.066090,0.065680,0.132643,-0.759413,-0.759081,-0.757810,-0.758171,-0.572895,1.841496,...,0.704287,0.635501,-1.146552,-0.648334,-0.684609,1.852881,1.910834,1.864024,1.415130,1.415130
2,-1.094682,-1.020275,-1.020534,-1.094666,-0.235245,-0.234923,-0.244280,-0.244682,1.745522,0.930416,...,0.858596,0.698016,-1.146552,-1.219359,-0.632636,0.935485,0.940820,0.941627,1.687780,1.687780
3,0.515575,0.494214,0.491069,0.509817,-0.035234,-0.034916,-0.034504,-0.038279,-0.572895,1.373707,...,0.481577,0.506401,-0.342278,-0.815871,-0.845328,1.304393,1.219656,1.280100,-0.793436,-0.793436
4,-0.843267,-0.857052,-0.857334,-0.843217,-0.368011,-0.367687,-0.371823,-0.372215,-0.572895,-1.016069,...,0.603815,0.578018,-0.668223,-0.689372,-0.766002,-1.017636,-0.857077,-1.125958,-0.307847,-0.307847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158605,-0.714566,-0.747346,-0.747643,-0.714500,-0.666304,-0.665974,-0.655440,-0.655809,-0.572895,-0.556844,...,0.597949,0.691112,-1.146552,-0.007785,-0.215929,-0.562086,-0.744124,-0.742777,1.469865,1.469865
158606,-0.286561,-0.329925,-0.327604,-0.292425,-0.742170,-0.736666,-0.739350,-0.739712,-0.572895,-0.981421,...,0.639539,0.699832,0.242535,-0.190947,-0.257830,-0.998210,-1.179486,-0.942536,0.961006,0.961006
158607,-0.472129,-0.592151,-0.592469,-0.472031,-0.286972,-0.286649,-0.301339,-0.301736,-0.572895,-1.628250,...,-0.014260,-0.286417,0.809548,1.195359,1.433088,-1.640920,-1.179486,-1.177999,0.964665,0.964665
158608,-0.448185,-0.511877,-0.512207,-0.454070,-0.642165,-0.641835,-0.645371,-0.645740,-0.572895,-0.614069,...,0.479543,0.544001,0.606969,0.203959,0.058581,-0.938463,-0.292405,-0.291202,0.962031,0.962031


In [128]:
xtrain.shape

(158610, 22)

In [129]:
XX=xtrain.copy()
XX['New_ID'] = range(0, 0+len(XX))
type(XX.columns.values)
cols = np.delete(XX.columns.values, np.where(XX.columns.values == 'New_ID'))
Y=XX.groupby(['New_ID']).apply(lambda subdf : subdf[cols].values).reset_index()
print('gropby done')
Z=Y[0].to_numpy()
A=np.stack(Z)
print('restack done ...transposing')
B=A.transpose(0,2,1)
B.shape

gropby done
restack done ...transposing


(158610, 22, 1)

In [131]:
dnnModel = models.Sequential()
# Layer 1 = input layer
# specify the input size in the first layer.
#------------------------------------------
# Layer 1 = input layer
dnnModel.add(layers.Conv1D(32, 3, activation="relu", input_shape=B.shape[1:3] ))

dnnModel.add(layers.MaxPooling1D((2)))


# Layer 2
dnnModel.add(layers.Conv1D(64, 3, activation="relu"))

dnnModel.add(layers.MaxPooling1D((2)))


# Layer 3
dnnModel.add(layers.Conv1D(64, 3, activation="relu" ))

dnnModel.add(layers.Flatten())
#------------------------------------------

#dnnModel.add(layers.Dense(64, activation='elu', input_shape= (10,20,)))
dnnModel.add(layers.Dense(20, activation='relu'))

# Layer 2 = hidden layer 
#dnnModel.add(layers.Dense(60, activation='relu'))

# Layer 3 = hidden layer 
#dnnModel.add(layers.Dense(30, activation='linear'))

# Layer 4 = output layer
dnnModel.add(layers.Dense(len(category2idx), activation='sigmoid'))

dnnModel.summary()
dnnModel.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'] )

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_22 (Conv1D)           (None, 20, 32)            128       
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 10, 32)            0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 8, 64)             6208      
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 4, 64)             0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 2, 64)             12352     
_________________________________________________________________
flatten_6 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 20)               

In [132]:

ytraintens=tf.convert_to_tensor(ytrain)

h  = dnnModel.fit( B, ytraintens, epochs=19, batch_size=10)

Epoch 1/19
15861/15861 [==============================] - 23s 1ms/step - loss: 1.3690 - accuracy: 0.5854
Epoch 2/19
15861/15861 [==============================] - 23s 1ms/step - loss: 0.9676 - accuracy: 0.7112
Epoch 3/19
15861/15861 [==============================] - 23s 1ms/step - loss: 0.8346 - accuracy: 0.7581
Epoch 4/19
15861/15861 [==============================] - 24s 1ms/step - loss: 0.7592 - accuracy: 0.7856
Epoch 5/19
15861/15861 [==============================] - 23s 1ms/step - loss: 0.7148 - accuracy: 0.8014
Epoch 6/19
15861/15861 [==============================] - 23s 1ms/step - loss: 0.6782 - accuracy: 0.8139
Epoch 7/19
15861/15861 [==============================] - 22s 1ms/step - loss: 0.6524 - accuracy: 0.8232
Epoch 8/19
15861/15861 [==============================] - 23s 1ms/step - loss: 0.6281 - accuracy: 0.8315
Epoch 9/19
15861/15861 [==============================] - 23s 1ms/step - loss: 0.6122 - accuracy: 0.8372
Epoch 10/19
15861/15861 [==============================